In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import the tabular dataset
drive_path = "/content/drive/MyDrive/MID.xlsx"

df = pd.read_excel(drive_path)

In [4]:
df.head(3)

,NAME,LINK,CONTAINS,INTRODUCTION,USES,BENEFITS,SIDE_EFFECT,HOW_TO_USE,HOW_WORKS,QUICK_TIPS,CHEMICAL_CLASS,HABIT_FORMING,THERAPEUTIC_CLASS,ACTION_CLASS,SAFETY_ADVICE_TO_ALCOHOL,SAFETY_ADVICE_TO_PREGNANCY,SAFETY_ADVICE_TO_BREAST_FEEDING,SAFETY_ADVICE_TO_DRIVING,SAFETY_ADVICE_TO_KIDNEY,SAFETY_ADVICE_TO_LIVER
0,Andol 0.5mg Tablet,https://www.1mg.com/drugs/andol-0.5mg-tablet-6...,Haloperidol (0.5mg),Andol 0.5mg Tablet can also be used for treati...,Schizophrenia,Schizophrenia\nSchizophrenia is a mental disor...,Most side effects do not require any medical a...,Take this medicine in the dose and duration as...,Andol 0.5mg Tablet is a typical antipsychotic....,Andol 0.5mg Tablet helps treat schizophrenia.\...,Butyrophenone Derivative,No,NEURO CNS,Typical Antipsychotics,Andol 0.5mg Tablet may cause excessive drowsin...,Andol 0.5mg Tablet may be unsafe to use during...,Andol 0.5mg Tablet is probably unsafe to use d...,"Andol 0.5mg Tablet may decrease alertness, aff...",Andol 0.5mg Tablet is safe to use in patients ...,Andol 0.5mg Tablet should be used with caution...
1,Avastin 100mg Injection,https://www.1mg.com/drugs/avastin-100mg-inject...,Bevacizumab (100mg),\nAvastin 100mg Injection is an effective medi...,Cancer of colon and rectum \nNon-small cell...,Cancer of colon and rectum\nCancer of the colo...,Most side effects do not require any medical a...,Your doctor or nurse will give you this medici...,Avastin 100mg Injection is a anti-angiogenic m...,"Avastin 100mg Injection is an effective, first...",Monoclonal antibody (mAb),No,ANTI NEOPLASTICS,Vascular endothelial growth factor (VEGF) inhi...,It is not known whether it is safe to consume ...,Avastin 100mg Injection may be unsafe to use d...,Avastin 100mg Injection is probably unsafe to ...,Avastin 100mg Injection may cause side effects...,Avastin 100mg Injection is probably safe to us...,Avastin 100mg Injection is probably safe to us...
2,Actorise 40 Injection,https://www.1mg.com/drugs/actorise-40-injectio...,Darbepoetin alfa (40mcg),Actorise 40 Injection is a medicine that needs...,Treatment of Anemia due to chronic kidney d...,Treatment of Anemia due to chronic kidney dise...,Most side effects do not require any medical a...,Your doctor or nurse will give you this medici...,Actorise 40 Injection is an erythropoiesis-sti...,Actorise 40 Injection helps in the treatment o...,"Amino Acids, Peptides Analogues",No,BLOOD RELATED,Erythropoiesis-stimulating agent (ESA),It is not known whether it is safe to consume ...,Actorise 40 Injection may be unsafe to use dur...,Actorise 40 Injection is probably unsafe to us...,Actorise 40 Injection does not usually affect ...,Actorise 40 Injection is safe to use in patien...,Actorise 40 Injection should be used with caut...


In [5]:
df.columns = df.columns.str.lower()
print(df.columns)

Index(['name', 'link', 'contains', 'introduction', 'uses', 'benefits',
       'side_effect', 'how_to_use', 'how_works', 'quick_tips',
       'chemical_class', 'habit_forming', 'therapeutic_class', 'action_class',
       'safety_advice_to_alcohol', 'safety_advice_to_pregnancy',
       'safety_advice_to_breast_feeding', 'safety_advice_to_driving',
       'safety_advice_to_kidney', 'safety_advice_to_liver'],
      dtype='object')


In [6]:
# Drop irrelevant columns
df.drop(columns=['link'], inplace=True)

In [7]:
df.isnull().sum()

,0
name,0
contains,0
introduction,11986
uses,0
benefits,0
side_effect,0
how_to_use,95
how_works,236
quick_tips,1463
chemical_class,91334


In [8]:
# Fill all NAs
df = df.fillna('no information available')
df.isnull().sum()

,0
name,0
contains,0
introduction,0
uses,0
benefits,0
side_effect,0
how_to_use,0
how_works,0
quick_tips,0
chemical_class,0


In [9]:
%%capture
!pip install langchain pypdf langchain_experimental faiss-cpu tiktoken langchain-huggingface sentence-transformers

In [10]:
import re
import unicodedata
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
def clean_text(text):
    if pd.isna(text):
        return text

    text = str(text).lower()

    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    text = re.sub(r'[^a-z0-9\s\.]', '', text)

    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [12]:
cols_to_clean = df.select_dtypes(include=['object']).columns.tolist()

for col in cols_to_clean:

    if col in df.columns:
        df[col] = df[col].apply(clean_text)

In [13]:
df.head(3)

,name,contains,introduction,uses,benefits,side_effect,how_to_use,how_works,quick_tips,chemical_class,habit_forming,therapeutic_class,action_class,safety_advice_to_alcohol,safety_advice_to_pregnancy,safety_advice_to_breast_feeding,safety_advice_to_driving,safety_advice_to_kidney,safety_advice_to_liver
0,andol 0.5mg tablet,haloperidol 0.5mg,andol 0.5mg tablet can also be used for treati...,schizophrenia,schizophrenia schizophrenia is a mental disord...,most side effects do not require any medical a...,take this medicine in the dose and duration as...,andol 0.5mg tablet is a typical antipsychotic....,andol 0.5mg tablet helps treat schizophrenia. ...,butyrophenone derivative,no,neuro cns,typical antipsychotics,andol 0.5mg tablet may cause excessive drowsin...,andol 0.5mg tablet may be unsafe to use during...,andol 0.5mg tablet is probably unsafe to use d...,andol 0.5mg tablet may decrease alertness affe...,andol 0.5mg tablet is safe to use in patients ...,andol 0.5mg tablet should be used with caution...
1,avastin 100mg injection,bevacizumab 100mg,avastin 100mg injection is an effective medici...,cancer of colon and rectum nonsmall cell lung ...,cancer of colon and rectum cancer of the colon...,most side effects do not require any medical a...,your doctor or nurse will give you this medici...,avastin 100mg injection is a antiangiogenic me...,avastin 100mg injection is an effective firstl...,monoclonal antibody mab,no,anti neoplastics,vascular endothelial growth factor vegf inhibitor,it is not known whether it is safe to consume ...,avastin 100mg injection may be unsafe to use d...,avastin 100mg injection is probably unsafe to ...,avastin 100mg injection may cause side effects...,avastin 100mg injection is probably safe to us...,avastin 100mg injection is probably safe to us...
2,actorise 40 injection,darbepoetin alfa 40mcg,actorise 40 injection is a medicine that needs...,treatment of anemia due to chronic kidney dise...,treatment of anemia due to chronic kidney dise...,most side effects do not require any medical a...,your doctor or nurse will give you this medici...,actorise 40 injection is an erythropoiesisstim...,actorise 40 injection helps in the treatment o...,amino acids peptides analogues,no,blood related,erythropoiesisstimulating agent esa,it is not known whether it is safe to consume ...,actorise 40 injection may be unsafe to use dur...,actorise 40 injection is probably unsafe to us...,actorise 40 injection does not usually affect ...,actorise 40 injection is safe to use in patien...,actorise 40 injection should be used with caut...


In [ ]:
df.to_csv('MID_processed.xlsx', index=False)

In [14]:
from langchain_core.documents import Document

In [15]:
# Convert dataset rows into document objects with metadata for efficient retrieval
medicine_docs = []
for _, row in df.iterrows():

    content = (
        f"medicine name: {row['name']}\n"
        f"chemical class: {row['chemical_class']}\n"
        f"therapeutic class: {row['therapeutic_class']}\n"
        f"action class: {row['action_class']}\n"
        f"composition: {row['contains']}\n"
        f"uses and benefits: {row['uses']} - {row['benefits']}\n"
        f"side effects: {row['side_effect']}\n"
        f"how it works: {row['how_works']}\n"
        f"quick tips: {row['quick_tips']}\n"
        f"alcohol safety advice: {row['safety_advice_to_alcohol']}, "
        f"pregnancy safety advice: {row['safety_advice_to_pregnancy']}, "
        f"driving safety advice: {row['safety_advice_to_driving']}"
    )

    metadata = {
        "source": "MDI_Dataset",
        "type": "Product_Sheet",
        "medicine_name": row['name']
    }
    medicine_docs.append(Document(page_content=content, metadata=metadata))

In [16]:
print(f"Procesados {len(medicine_docs)} documentos.")

Procesados 192807 documentos.


In [17]:
medicine_docs[10]

Document(metadata={'source': 'MDI_Dataset', 'type': 'Product_Sheet', 'medicine_name': 'azacytin injection'}, page_content='medicine name: azacytin injection\nchemical class: pyrimidine nucleoside analogue\ntherapeutic class: anti neoplastics\naction class: antimetabolites\ncomposition: azacitidine 100mg\nuses and benefits: blood cancer treatment of myelodysplastic syndrome - blood cancer blood cancer also called leukemia is a cancer of the bloodforming tissues that decreases the bodys ability to fight infection. azacytin injection kills or stops the growth of cancer cells and also prevents the multiplication of cancer cells. it is a potent and very toxic medicine and you should discuss the risks and benefits with your doctor. you should avoid drinking alcohol while having this treatment and drink plenty of water to stay hydrated. treatment of myelodysplastic syndrome azacytin injection is used to treat myelodysplastic syndrome a group of conditions in which the bone marrow produces blo

In [18]:
%%capture
!pip install langchain-huggingface sentence-transformers faiss-cpu

In [19]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
# Initialize the embedding model on GPU (CUDA)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [21]:
# Create and save the FAISS vector database locally for persistent storage
vectorstore = FAISS.from_documents(medicine_docs, embeddings)
vectorstore.save_local("faiss_mid")

In [22]:
query = "What is the mechanism of action and side effects of Azithromycin?"

print(f"Consulta: '{query}'\n")

results = vectorstore.similarity_search(query, k=5)

for i, doc in enumerate(results):

    medicine_name = doc.metadata.get('medicine_name', 'Unknown')

    print(f" RESULTADO {i+1}")
    print(f" MEDICAMENTO: {medicine_name}")

    print(f" DETALLES DE LA FICHA:\n{doc.page_content[:500]}...")
    print("-" * 60, "\n")

if len(results) > 0:
    print(f"ÉXITO: Se han recuperado {len(results)} registros del dataset MID relevantes.")
else:
    print("ADVERTENCIA: No se encontraron resultados. Asegúrate de haber ejecutado 'FAISS.from_documents(medicine_docs, embeddings)'.")

Consulta: 'What is the mechanism of action and side effects of Azithromycin?'

 RESULTADO 1
 MEDICAMENTO: azithromycin 100mg syrup
 DETALLES DE LA FICHA:
medicine name: azithromycin 100mg syrup
chemical class: macrolides
therapeutic class: anti infectives
action class: macrolides
composition: azithromycin 100mg
uses and benefits: treatment of bacterial infections - treatment of bacterial infections azithromycin 100mg syrup is an antibiotic medicine that can be used to treat many different infections caused by bacteria. these include infections of the blood brain lungs bones joints urinary tract stomach and intestines. it can also be used to treat...
------------------------------------------------------------ 

 RESULTADO 2
 MEDICAMENTO: azithromycin 250mg tablet
 DETALLES DE LA FICHA:
medicine name: azithromycin 250mg tablet
chemical class: macrolides
therapeutic class: anti infectives
action class: macrolides
composition: azithromycin 250mg
uses and benefits: treatment of bacterial i